<a href="https://colab.research.google.com/github/RRADJon/pyPBPK_dev/blob/main/IO_ChemPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🧪 PBPK Parameter Calculator (Upload/Download)

!pip install rdkit pandas

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from google.colab import files
import io

# 1. Function to process chemical logic
def calculate_pbpk_row(smiles, melting_point_c=150):
    if pd.isna(smiles) or not isinstance(smiles, str):
        return [None] * 5

    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return [None] * 5

    # Physicochemical Descriptors
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    tpsa = Descriptors.TPSA(mol)

    # Permeability (Peff) Prediction (cm/s -> cm/min)
    log10_peff_cms = -3.4 + (0.18 * logp) - (0.009 * tpsa) - (0.0006 * mw)
    peff_cm_min = (10**log10_peff_cms) * 60

    # Solubility Prediction (GSE Method)
    log_s = 0.5 - logp - 0.01 * (melting_point_c - 25)
    solubility_mg_ml = (10**log_s) * mw

    return [
        round(mw, 2),
        round(logp, 2),
        round(tpsa, 2),
        round(solubility_mg_ml, 4),
        round(peff_cm_min, 6)
    ]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 22.1 MB/s eta 0:00:00


In [ ]:
# 2. Upload the Excel file
print("Please upload your Excel file (.xlsx or .xls):")
uploaded = files.upload()

Please upload your Excel file (.xlsx or .xls):


Saving Cleaned_PK_mw_logP_oral_iv.xlsx to Cleaned_PK_mw_logP_oral_iv.xlsx


In [ ]:
for filename in uploaded.keys():
    # 3. Read the uploaded file
    df = pd.read_excel(io.BytesIO(uploaded[filename]))
    print(f"Processing {filename}...")

    # 4. Perform calculations (Update 'Smiles-PUBCHEM' if your column name is different)
    smiles_col = 'Smiles-PUBCHEM'
    if smiles_col not in df.columns:
        print(f"Error: Column '{smiles_col}' not found in file.")
    else:
        results = df[smiles_col].apply(lambda x: pd.Series(calculate_pbpk_row(x)))
        results.columns = ["MW (g/mol)", "LogP", "TPSA (Å²)", "Solubility (mg/mL)", "Peff (cm/min)"]

        # Merge results with original data
        df_output = pd.concat([df, results], axis=1)

        # 5. Export and Download
        output_filename = f"Processed_{filename}"
        df_output.to_excel(output_filename, index=False)
        print(f"Done! Downloading {output_filename}...")
        files.download(output_filename)

Processing Cleaned_PK_mw_logP_oral_iv.xlsx...
Done! Downloading Processed_Cleaned_PK_mw_logP_oral_iv.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>